In [1]:
!pip install pymongo
!pip install razdel
!pip install pymorphy2
!pip install slovnet
!pip install geopy
!pip install geolocator

In [2]:
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
from razdel import tokenize
from razdel import sentenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import os
from bson import ObjectId
from navec import Navec
from slovnet import NER
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="k")
ner = NER.load('slovnet_ner_news_v1.tar')
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner.navec(navec)

NER(
    infer=NERInfer(
        model=NER(
            emb=WordShapeEmbedding(
                word=NavecEmbedding(
                    id='news_v1_1B_250K_300d_100q',
                    indexes=Weight(
                        shape=[250002,
                         100],
                        dtype='uint8',
                        array=array([[176, 222, 248, ..., 244, 183, 191],
                               [215, 200, 168, ..., 120, 217,  21],
                               [ 83, 174,  54, ..., 106,  88, 251],
                               ...,
                               [133, 125, 123, ..., 124,  94,  24],
                               [183,  49, 180, ..., 151, 167,  68],
                               [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)
                    ),
                    codes=Weight(
                        shape=[100,
                         256,
                         3],
                        dtype='float32',
                        array

In [4]:
client = pymongo.MongoClient('localhost', 27017, unicode_decode_error_handler='ignore')
db = client['mongo_for_folklore']
text_collection = db.texts # инфа о самом тексте
lemmas = db.lemmas # инфа о лемме и токене
dictionary = db.dictionary
plots = db.plots # доп разметка
print(client.test_database)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, unicode_decode_error_handler='ignore'), 'test_database')


In [41]:
lemmas.create_index([('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)
dictionary.create_index([('token', pymongo.ASCENDING), ('iniForm', pymongo.ASCENDING), ('tags', pymongo.ASCENDING)], unique=True)

'token_1_iniForm_1_tags_1'

In [40]:
# lemmas.drop_index('iniForm_1_tags_1_forms')
# for x in lemmas.list_indexes():
#     print(x)


# запустить, если ошибка с дублированием в базе данных. удаляет все индексы, потом их надо переустановить
dictionary.drop_indexes() 

In [6]:
def putFolkloreData(file_name):

    file_name = file_name.replace('.txt', '')

    try:
        data = data_file.loc[file_name.replace('~', ' ')]
        doc = {'doc_text': data['Text'],
               'genre': data['Genre'],
               'info': data['Info'],
               'comment': data['Comment'],
               'collector': data['Collector'],
               'typewriter': data['Typewriter']}
        indx_doc = text_collection.insert_one(doc).inserted_id


        suj_annotation = []
        path = 'data/annotation/' + file_name + '.txt/demo.xml'
        with open(path, 'r', encoding='utf-8') as f:
            xml_doc = f.read()
        soup = BeautifulSoup(xml_doc, 'xml')
        for pers_span in soup.Document.find_all('custom.Span'):  # ищет сюжеты
            suj_annotation.append({'text': pers_span.text.replace('\n', ' '), 
                                   'hero_ind': pers_span['label_ind'],
                                   'hero_type': pers_span['label'], 
                                  'id_doc': indx_doc})
        for suj_span in soup.Document.find_all('webanno.custom.Sujet'):
            suj_annotation.append({'text': suj_span.text.replace('\n', ' '), 
                                   'label' : suj_span['type_s'], 
                                   'end' : int(suj_span['end']), 
                                   'begin' : int(suj_span['begin']), 
                                  'id_doc': indx_doc})
        if suj_annotation != []:
            indx_plot = plots.insert_many(suj_annotation).inserted_ids


        sentences = list(sentenize(doc['doc_text']))
        for sent in [_.text for _ in sentences]:
            for token in [_.text for _ in tokenize(sent)]:

                word_parsed = morph.parse(token)[0]
                norm_f = word_parsed.normal_form
                tags = word_parsed.tag
                if lemmas.count_documents({"iniForm": norm_f, "POS": tags.POS}) == 0:
#                     , "forms":[{'token' : token, 'tags' : str(tags)}]
                    lemma_id = lemmas.insert_one({"iniForm": norm_f, "POS": tags.POS, "freq": 1,  "docs": [indx_doc]}).inserted_id
                else:
                    inis = lemmas.find({"iniForm": norm_f, "POS": tags.POS})
                    lemma_id = inis[0]["_id"]
                    lemmas.find_one_and_update({"_id": lemma_id},
                                               {"$inc": {"freq": 1},
                                                "$addToSet": {"docs": indx_doc}})
#                     if lemmas.count_documents({'forms.token' : token, 'forms.tags' : str(tags), "POS": tags.POS}) == 0:
#                         lemmas.find_one_and_update({"_id": lemma_id},
#                                                    { "$addToSet": {"forms": {'token' : token, 'tags' : str(tags)}}})
                
#                #для токенов
                if dictionary.count_documents({"token":token, "iniForm": norm_f, "tags": str(tags)}) == 0:
                    wf_id = dictionary.insert_one({"token":token, 
                                                   "iniForm": norm_f, 
                                                   "tags": str(tags), 
                                                   "freq": 1, 
                                                   "docs": [indx_doc]}).inserted_id
                else:
                    wrdf = dictionary.find({"token":token, "iniForm": norm_f, "tags": str(tags)})
                    wf_id=wrdf[0]["_id"]
                    dictionary.find_one_and_update({"_id": wf_id}, {"$inc":  {"freq": 1}}) 
                    dictionary.find_one_and_update({"_id": wf_id}, {"$addToSet": {"docs": indx_doc}}) 


    
    except KeyError:
        print('Failed to get file: ', file_name)

In [6]:
def de(text):
    return text.replace('~', ' ').replace('.docx', '').replace('.txt', '')


data_file = pd.read_csv('corpus_UPD - corpus_UPD.csv',  encoding = 'utf-8') #index_col='Filename',

data_file['Filename'] = data_file['Filename'].apply(de)
data_file = data_file.set_index('Filename')
data_file.fillna('').head(1)

,Пословицы и поговорки.,Number,Genre,Info,Comment,Text,Collector,Typewriter,RawText,empty_text,перезалить,Unnamed: 12
Filename,,,,,,,,,,,,
фольклорный архив-3 100. Частушка. С горки на горку...,0.0,100,Частушка,Кирпичникова Клавдия Федоровна\nИркутская обла...,,"С горки на горку,\nЕще на камешок\nСкорее пода...",Тубол Л. 1 группа. 1980 г,Рыбинская Дарья.,"С горки на горку,\nЕще на камешок\nСкорее пода...",FALSE,,


In [8]:
for x in os.listdir('data/annotation/')[:10]:
    if 'demo.xml' in os.listdir('data/annotation/' + x):
        putFolkloreData(x)

In [7]:
for x in text_collection.find().limit(1):
    print(x)
    
    

{'_id': ObjectId('648cf9c561b224ea1ad2d3f3'), 'doc_text': 'Старый год-то распрощался,\nНавсегда в тайгу подался,\nСтали новый год встречать, \nДиков вздумал догонять.\nТоня парень-то не трус, \nОн мигом сел на Белорус,\nПрибавлял он обороты\nИ провалился в сухолёд,\nПолучился тут обвал,\nА то бы старый год догнал.', 'genre': 'Стихотворение.', 'info': 'Самодуров Фёдор Владимирович, 1902 г. село Круплянка.', 'comment': 'Стихи, сочинённые самими сказителями.', 'collector': nan, 'typewriter': 'Троицкая А.А., 18.06.2022.'}


In [35]:
for x in plots.find().limit(1):
    print(x)

{'_id': ObjectId('648cfa8661b224ea1ad2d3f5'), 'text': ' Я ', 'hero_ind': '0', 'hero_type': 'hm', 'id_doc': ObjectId('648cfa8661b224ea1ad2d3f4')}


In [9]:
for x in lemmas.find().limit(1):
    print(x)

{'_id': ObjectId('648cfa8661b224ea1ad2d415'), 'iniForm': 'двадцать', 'POS': 'NUMR', 'freq': 4, 'docs': [ObjectId('648cfa8661b224ea1ad2d3f4'), ObjectId('648f7d14adadd388c44683b8'), ObjectId('648f7dc4adadd388c44683d9'), ObjectId('649024a86c09d7623df0718b')], 'forms': [{'token': 'Двадцать', 'tags': 'ЧИСЛ им'}]}


In [10]:
for x in dictionary.find().limit(10):
    print(x)

{'_id': ObjectId('649024a86c09d7623df071ac'), 'token': 'Двадцать', 'iniForm': 'двадцать', 'tags': 'NUMR nomn', 'freq': 1, 'docs': [ObjectId('649024a86c09d7623df0718b')]}
{'_id': ObjectId('649024a86c09d7623df071ad'), 'token': 'шестого', 'iniForm': 'шесть', 'tags': 'ADJF,Anum masc,sing,gent', 'freq': 1, 'docs': [ObjectId('649024a86c09d7623df0718b')]}
{'_id': ObjectId('649024a86c09d7623df071ae'), 'token': 'декабря', 'iniForm': 'декабрь', 'tags': 'NOUN,inan,masc sing,gent', 'freq': 2, 'docs': [ObjectId('649024a86c09d7623df0718b')]}
{'_id': ObjectId('649024a86c09d7623df071af'), 'token': 'сорок', 'iniForm': 'сорок', 'tags': 'NUMR nomn', 'freq': 1, 'docs': [ObjectId('649024a86c09d7623df0718b')]}
{'_id': ObjectId('649024a86c09d7623df071b0'), 'token': 'первого', 'iniForm': 'первое', 'tags': 'NOUN,inan,neut sing,gent', 'freq': 1, 'docs': [ObjectId('649024a86c09d7623df0718b')]}
{'_id': ObjectId('649024a86c09d7623df071b1'), 'token': 'года', 'iniForm': 'год', 'tags': 'NOUN,inan,masc sing,gent', 'fr

In [14]:
# найди быть в прошедшем времени, ед числе 

for x in dictionary.find({'$and':[{'tags': {'$regex': 'past'}}, {'tags': {'$regex': 'sing'}}, {'iniForm':'быть'}]}):
    print(x)

{'_id': ObjectId('64900ae9397b9020042fa35b'), 'token': 'был', 'iniForm': 'быть', 'tags': 'VERB,impf,intr masc,sing,past,indc', 'freq': 7, 'docs': [ObjectId('64900ae9397b9020042fa32b'), ObjectId('64900af6397b9020042fa6f8'), ObjectId('64900af8397b9020042fa79f')]}
{'_id': ObjectId('64900aeb397b9020042fa3e4'), 'token': 'Был', 'iniForm': 'быть', 'tags': 'VERB,impf,intr masc,sing,past,indc', 'freq': 2, 'docs': [ObjectId('64900ae9397b9020042fa32b')]}
{'_id': ObjectId('64900aec397b9020042fa43e'), 'token': 'было', 'iniForm': 'быть', 'tags': 'VERB,impf,intr neut,sing,past,indc', 'freq': 5, 'docs': [ObjectId('64900ae9397b9020042fa32b')]}
{'_id': ObjectId('64900aef397b9020042fa506'), 'token': 'была', 'iniForm': 'быть', 'tags': 'VERB,impf,intr femn,sing,past,indc', 'freq': 3, 'docs': [ObjectId('64900ae9397b9020042fa32b'), ObjectId('64900af6397b9020042fa6f8')]}
{'_id': ObjectId('64900af3397b9020042fa64a'), 'token': 'Была', 'iniForm': 'быть', 'tags': 'VERB,impf,intr femn,sing,past,indc', 'freq': 1, '

In [23]:
# использую ИЛИ и И в одном запросе
# найди документы в которых (И) лемма быть стоит в прошедшем времени (ИЛИ) единственном числе

for x in dictionary.find({'$and': [{'$or':[{'tags': {'$regex': 'past'}}, {'tags': {'$regex': 'sing'}}]}, {'iniForm':'быть'}]}):
    print(x)

{'_id': ObjectId('649024a86c09d7623df071b3'), 'token': 'был', 'iniForm': 'быть', 'tags': 'VERB,impf,intr masc,sing,past,indc', 'freq': 6, 'docs': [ObjectId('649024a86c09d7623df0718b'), ObjectId('649024ad6c09d7623df073c5')]}
{'_id': ObjectId('649024a96c09d7623df071fc'), 'token': 'Был', 'iniForm': 'быть', 'tags': 'VERB,impf,intr masc,sing,past,indc', 'freq': 2, 'docs': [ObjectId('649024a86c09d7623df0718b')]}
{'_id': ObjectId('649024a96c09d7623df0722c'), 'token': 'было', 'iniForm': 'быть', 'tags': 'VERB,impf,intr neut,sing,past,indc', 'freq': 5, 'docs': [ObjectId('649024a86c09d7623df0718b')]}
{'_id': ObjectId('649024aa6c09d7623df072a5'), 'token': 'была', 'iniForm': 'быть', 'tags': 'VERB,impf,intr femn,sing,past,indc', 'freq': 3, 'docs': [ObjectId('649024a86c09d7623df0718b'), ObjectId('649024ad6c09d7623df073c5')]}
{'_id': ObjectId('649024ab6c09d7623df072f4'), 'token': 'были', 'iniForm': 'быть', 'tags': 'VERB,impf,intr plur,past,indc', 'freq': 4, 'docs': [ObjectId('649024a86c09d7623df0718b'

In [149]:
# удалить все
# text_collection.delete_many({})
# lemmas.delete_many({})
# plots.delete_many({})
# dictionary.delete_many({})

In [375]:
# удалить определенные документы
#  plots.delete_many({'hero_type': 'wm'})

In [376]:
# удалить документ по id 
# plots.delete_one({'_id': ObjectId('6488bd6351c58258acc7f5f2')})

In [377]:
# обновить разметку

# print(plots.find_one_and_update({'_id': ObjectId('6488bd6351c58258acc7f5f4')}, {'$set' :{'hero_type': 'hj'}}))
# plots.find({'_id' : ObjectId('6488bd6351c58258acc7f5f4')})[0]

In [24]:
for x in text_collection.find({'$and':[{'genre' : {'$regex': '[Чч]астушк.*'}}, {'info': {'$regex': '\sг\.\s[А-Я][а-я]+'}}]}).limit(2):
    print(x)
    print('----------')

{'_id': ObjectId('648cfa9161b224ea1ad2d622'), 'doc_text': 'На горе стоит пекарня,\nДедка сушит сухари.\nПриходите на вечерку -\nМы наставим фонари.', 'genre': 'Частушки', 'info': 'Назарова Анна Федотьевна\nНациональность: русская\nГод рождения: 1906г.\nМесто рождения: г. Верхнеудинск\nОбразование: неграмотная\nПрофессия: пенсионерка', 'comment': nan, 'collector': 'студентами 1 курса 3 группы филологического ф – та ИГУ Съяновой А., Тирских Н. 11 июля 1981г. Место записи: с. Бальзино Дульдургинского р – на Читинской обл.', 'typewriter': 'студентка 1 курса группы 211121 – ЗБ  филологического ф-та ИГУ Иванова Ульяна 2019 г. Место: птг. Тыреть, мкр Солерудник.'}
----------
{'_id': ObjectId('648cfa9161b224ea1ad2d62d'), 'doc_text': 'На горе-горушке\nЖили две старушки.\nСтарик старуху за виски\n«Не разбрасывай куски».', 'genre': 'Частушки', 'info': 'Назарова Анна Федотьевна\nНациональность: русская\nГод рождения: 1906г.\nМесто рождения: г. Верхнеудинск\nОбразование: неграмотная\nПрофессия: пен

In [199]:
# s = set()
b = set()
b.update('5')
s.update('5')
s.intersection(b)

{'5'}

In [ ]:
def get_by_plot(response_list):
    plot = set()
    for x in plots.find({'label': {'$in': response_plot}}):
        plot.update(text_collection.find({"_id": x['id_doc']})[0]['_id'])
    return plot

In [19]:
def preprocess(token):
    token = morph.parse(token.strip())[0].normal_form
    return token


def get_doc_ids(exact: str,
                lemma: str, 
             response_tags: list, 
             response_hero: list, 
             response_plot: list, 
            response_genre='',
            response_info=''):
    """ все кроме леммы идет в forms.tags и подается в списке response_tags """
    
    res = []

    # если есть точная подстрока, то на параметры не смотрим
    if exact: 
        for x in text_collection.find({'doc_text': {'$regex': exact}}):
            res.append(x['_id'])
        return res
    
    
    list_of_tags = []
    if response_tags:
        list_of_tags = [{'forms.tags': re.compile(i)} for i in response_tags]
    if lemma: 
        list_of_tags.append({'iniForm': preprocess(lemma)})
    if list_of_tags:
        for x in lemmas.find({'$and': list_of_tags}):
            for doc in text_collection.find({'_id': {'$in': x['docs']}}):
                res.append(doc['_id'])
#                 print(doc)

    if response_hero:
        hero = []
        for y in plots.aggregate([{'$match' : {'hero_type': {'$in': response_hero}}}, {'$group': {'_id': '$id_doc'}}]):
            hero.append(text_collection.find({'_id': y['_id']})[0]['_id'])
                
    if response_plot:
#         print(response_plot)
        plot = []
        for x in plots.find({'label': {'$in': response_plot}}):
            plot.append(text_collection.find({"_id": x['id_doc']})[0]['_id'])
            
        plot = []
        for x in plots.find({'$and':[{'genre' : {'$regex': response_genre}}, 
                                     {'info': {'$regex': response_info}}]}):
            info.append(text_collection.find({"_id": x['id_doc']})[0]['_id'])
    return res

res = get_doc_ids('горе', 'гора', [],['hw', 'dw', 'vw'], ['+s3'], 'част')
res

[ObjectId('64902f34f179e19ef227d40d')]

In [150]:
for x in text_collection.find({'_id': ObjectId('64902f34f179e19ef227d40d')}).limit(1):
    print(x)

{'_id': ObjectId('64902f34f179e19ef227d40d'), 'doc_text': 'На горе стоит пекарня,\nДедка сушит сухари.\nПриходите на вечерку -\nМы наставим фонари.', 'genre': 'Частушки', 'info': 'Назарова Анна Федотьевна\nНациональность: русская\nГод рождения: 1906г.\nМесто рождения: г. Верхнеудинск\nОбразование: неграмотная\nПрофессия: пенсионерка', 'comment': nan, 'collector': 'студентами 1 курса 3 группы филологического ф – та ИГУ Съяновой А., Тирских Н. 11 июля 1981г. Место записи: с. Бальзино Дульдургинского р – на Читинской обл.', 'typewriter': 'студентка 1 курса группы 211121 – ЗБ  филологического ф-та ИГУ Иванова Ульяна 2019 г. Место: птг. Тыреть, мкр Солерудник.'}


In [163]:
for x in plots.find():
    print(x)

{'_id': ObjectId('64900ae9397b9020042fa32c'), 'text': ' Я ', 'hero_ind': '0', 'hero_type': 'hm', 'id_doc': ObjectId('64900ae9397b9020042fa32b')}
{'_id': ObjectId('64900ae9397b9020042fa32d'), 'text': ' матери ', 'hero_ind': '1', 'hero_type': 'hw', 'id_doc': ObjectId('64900ae9397b9020042fa32b')}
{'_id': ObjectId('64900ae9397b9020042fa32e'), 'text': ' я ', 'hero_ind': '0', 'hero_type': 'hm', 'id_doc': ObjectId('64900ae9397b9020042fa32b')}
{'_id': ObjectId('64900ae9397b9020042fa32f'), 'text': ' Она ', 'hero_ind': '1', 'hero_type': 'hw', 'id_doc': ObjectId('64900ae9397b9020042fa32b')}
{'_id': ObjectId('64900ae9397b9020042fa330'), 'text': ' её ', 'hero_ind': '1', 'hero_type': 'hw', 'id_doc': ObjectId('64900ae9397b9020042fa32b')}
{'_id': ObjectId('64900ae9397b9020042fa331'), 'text': ' милиция ', 'hero_ind': '2', 'hero_type': 'vm', 'id_doc': ObjectId('64900ae9397b9020042fa32b')}
{'_id': ObjectId('64900ae9397b9020042fa332'), 'text': ' её ', 'hero_ind': '1', 'hero_type': 'hw', 'id_doc': ObjectId

In [26]:
#моя функция

def add_catecory_to_query(category_list, tag_specific):
    sub_query_and = {'$and': []}
    for category in category_list:
        sub_query_or = {'$or': []}
        for tag_cat in category:
            sub_query_or['$or'].append({tag_specific: {'$regex': tag_cat}})
        if sub_query_or['$or'] != []:
            sub_query_and['$and'].append(sub_query_or)
    return sub_query_and

def get_doc_ids(exact: str,
                lemma: str, 
             response_tags: list, 
             response_hero: list, 
             response_plot: list, 
            response_genre='',
            response_info=''):
    
    res_docs = []
    
    query = []
    if exact != '':
        query.append({'token':exact})
    if lemma != '':
        query.append({'iniForm':lemma})

    # запрос для морфологии     
    if response_tags != [[], [], [], [], []]:
        query.append(add_catecory_to_query(response_tags, 'tags'))
    
    
    if query:
        for x in dictionary.find({'$and': query}):
            res_docs+= x['docs']
    
    
    query_plot = []
    # запрос для героев
    if response_hero != [[], [], [], [], [], []]:
        query_plot += add_catecory_to_query(response_hero, 'hero_type')['$and']
        
    #запрос для сюжетов
    if response_plot != [[], [], []]:
        query_plot += add_catecory_to_query(response_plot, 'label')['$and']

    
    # запрос для инфо и жанра     
    query_info_genre = []
    if response_info != '':
        query_info_genre.append({'info' : {'$regex': response_info}})
        
    if response_genre != '':
        query_info_genre.append({'genre' : {'$regex': response_genre}})
    
    res_docs_plot = []
    if query_info_genre != []:
#         res_docs_plot = []
        for x in text_collection.find({'$and': query_info_genre}):
#             print('!!!', x)
            res_docs_plot.append(x['_id'])
        

        
    result_query = []
    if query != []:
        result_query.append({'id_doc': {'$in': res_docs}})
    if query_plot != []:
        result_query.append({'$and': query_plot})
    if res_docs_plot != []:
        result_query.append({'id_doc': {'$in': res_docs_plot}})
                
    DOCS_FINAL = []        
                
    if result_query != []:
        pipeline_query =  [{'$match' : {'$and': result_query}}, {'$group': {'_id': '$id_doc'}}]
#         print(pipeline_query)
        for x in plots.aggregate(pipeline_query):
#             print(x, '!!!')
            for y in text_collection.find({"_id": x['_id']}):
                DOCS_FINAL.append(y['_id'])
                print(', !!', y)
                
    return DOCS_FINAL
    
    
    
u = get_doc_ids('старый', '', 
            [[], [], [], [], []],
            [[], [], [], [], [], []],
             [[], [], []], 
            '', 
           '')
u

[]

In [42]:
####### проверка леммы

#моя функция

def add_catecory_to_query(category_list, tag_specific):
    sub_query_and = {'$and': []}
    for category in category_list:
        sub_query_or = {'$or': []}
        for tag_cat in category:
            sub_query_or['$or'].append({tag_specific: {'$regex': tag_cat}})
        if sub_query_or['$or'] != []:
            sub_query_and['$and'].append(sub_query_or)
    return sub_query_and


def get_doc_ids(exact: str,
                lemma: str, 
             response_tags: list, 
             response_hero: list, 
             response_plot: list, 
            response_genre='',
            response_info=''):
    
    res_docs = []
    
    query = []
    if exact != '':
        query.append({'token':exact})
    if lemma != '':
        query.append({'iniForm':lemma})

    # запрос для морфологии     
    if response_tags != [[], [], [], [], []]:
        query.append(add_catecory_to_query(response_tags, 'tags'))
    
    
    if query:
        for x in dictionary.find({'$and': query}):
            res_docs+= x['docs']
    
    
    query_plot = []
    # запрос для героев
    if response_hero != [[], [], [], [], [], []]:
        query_plot += add_catecory_to_query(response_hero, 'hero_type')['$and']
        
    #запрос для сюжетов
    if response_plot != [[], [], []]:
        query_plot += add_catecory_to_query(response_plot, 'label')['$and']

    
    # запрос для инфо и жанра     
    query_info_genre = []
    if response_info != '':
#         print(response_info)
        query_info_genre.append({'info' : {'$regex': response_info}})
        
    if response_genre != '':
        query_info_genre.append({'genre' : {'$regex': response_genre}})
    
    res_docs_plot = []
    if query_info_genre != []:
        for x in text_collection.find({'$and': query_info_genre}):
#             print('!!!', x)
            res_docs_plot.append(x['_id'])
    
        
    result_query = []
    if query != []:
        result_query.append({'id_doc': {'$in': res_docs}})
    if query_plot != []:
        result_query.append({'$and': query_plot})
    if res_docs_plot != []:
#         print(res_docs_plot)
        result_query.append({'id_doc': {'$in': res_docs_plot}})
                
    DOCS_FINAL = []        
                
    if result_query != []:
        pipeline_query =  [{'$match' : {'$and': result_query}}, {'$group': {'_id': '$id_doc'}}]
#         print(pipeline_query)
        for x in plots.aggregate(pipeline_query):
#             print(x, '!!!')
            for y in text_collection.find({"_id": x['_id']}):
                DOCS_FINAL.append(y['_id'])
                print(', !!', y)
                
    return DOCS_FINAL
    
    
    
u = get_doc_ids('', 'милый', 
            [[], [], [], [], []],
            [[], [], [], [], [], []],
             [[], [], []], 
            '', 
           '')
u

, !! {'_id': ObjectId('649024ad6c09d7623df0741e'), 'doc_text': 'Мне мой милый изменил,\nНа козе уехал в Крым,\nА я маху не дала,\nНа корове догнала.', 'genre': 'Частушка.\nМотив измены.', 'info': 'Рыжов Иван Иванович.\nГород Иркутск, улица Баумана, дом 239, квартира 55.\n1920 год рождения.\nВетеран Великой Отечественной войны, пенсионер.', 'comment': nan, 'collector': 'Гринштейн', 'typewriter': 'Лебедева З.М., 25.06.2022'}


[ObjectId('649024ad6c09d7623df0741e')]

In [27]:
def get_doc_ids(exact: str,
                lemma: str, 
             response_tags: list, 
             response_hero: list, 
             response_plot: list, 
            response_genre='',
            response_info=''):
    
    res_docs = []
    
    query = []
    if exact != '':
        query.append({'token':exact})
    if lemma != '':
        query.append({'iniForm':lemma})

    # запрос для морфологии     
    if response_tags != [[], [], [], [], []]:
        query.append(add_catecory_to_query(response_tags, 'tags'))
    
    
    if query:
        for x in dictionary.find({'$and': query}):
            res_docs += x['docs']
    
    
    query_plot = []
    # запрос для героев
    if response_hero != [[], [], [], [], [], []]:
        query_plot += add_catecory_to_query(response_hero, 'hero_type')['$and']
        
    #запрос для сюжетов
    if response_plot != [[], [], []]:
        query_plot += add_catecory_to_query(response_plot, 'label')['$and']

    
    # запрос для инфо и жанра     
    query_info_genre = []
    if response_info != '':
        query_info_genre.append({'info' : {'$regex': response_info}})
        
    if response_genre != '':
        query_info_genre.append({'genre' : {'$regex': response_genre}})
    
    
    res_docs_plot = []
    if query_info_genre != []:
        for x in text_collection.find({'$and': query_info_genre}):
#             print('!!!', x)
            res_docs_plot.append(x['_id'])
        
        
            
    result_query = []
    if query != []:
        result_query.append({'id_doc': {'$in': res_docs}})
    if query_plot != []:
        result_query.append({'$and': query_plot})
    if res_docs_plot:
        result_query.append({'id_doc': {'$in': res_docs_plot}})
                
    DOCS_FINAL = []        
                
    if result_query != []:
        pipeline_query =  [{'$match' : {'$and': result_query}}, {'$group': {'_id': '$id_doc'}}]
        print(pipeline_query)
        for x in plots.aggregate(pipeline_query):
#             print(x, '!!!')
            for y in text_collection.find({"_id": x['_id']}):
                DOCS_FINAL.append(y['_id'])
# =============================================================================
#                 print(', !!', y)
# =============================================================================
                
    return DOCS_FINAL

u = get_doc_ids('старый', '', 
            [[], [], [], [], []],
            [[], [], [], [], [], []],
             [[], [], []], 
            '', 
           '')
u

[{'$match': {'$and': [{'id_doc': {'$in': []}}]}}, {'$group': {'_id': '$id_doc'}}]


[]

In [19]:
u = get_doc_ids('Милиционеру', 
            'милиционер', 
            (['NOUN', 'ADJF'], ['nomn', 'loct', 'dat'], [], [], []),
            ([], [], [], [], [], []),
             ([], [], ['\+r']), 
            '', 
           'Котов Владимир Васильевич.\nРодился в 1923 году в селе Гришево Черемховского района Иркутской области.')
u


[ObjectId('649024a86c09d7623df0718b')]

In [243]:
for x in plots.aggregate([{'$match': {'$and': [{'id_doc': {'$in': [ObjectId('64900ae9397b9020042fa32b')]}}, {'$and': [{'$or': [{'label': {'$regex': '\\+r'}}]}]}, {'id_doc': {'$in': [ObjectId('64900ae9397b9020042fa32b')]}}]}}, {'$group': {'_id': '$id_doc'}}]):
    print(x)
    for y in text_collection.find({"_id": x['_id']}):
        print(', !!', y)

{'_id': ObjectId('64900ae9397b9020042fa32b')}
, !! {'_id': ObjectId('64900ae9397b9020042fa32b'), 'doc_text': 'Двадцать шестого декабря сорок первого года я был взят в армию. А, короче, надо начинать с того, что я здесь родился, на Гришево я здесь родился. И родился я, вон дом стоит вот, я хочу вам показать… На задах моего огорода стоит барак, вот, в 23-м году я в этом бараке родился. Это, значит, моя родина.\n– Как ваше отчество?\n– Васильевич, Владимир Васильевич.\n   Учился я в 17-ой школе. А потом в 36-ом году или в 35-ом году уехал к отцу. Мать с отцом разошлися, А я жил с бабушкой и дедушкой. А мать уехала. А я с бабушкой жил до ухода в армию. 26 декабря 41-го года был призван в армию и служил в Чите в Забайкалье. Был направлен в училище, в радио училище. Это радио училище в 42-м году я кончил. Потом был направлен в Забайкалье. Станция Борзя на Маньчжурской границе. После чего в феврале месяце 43-го года маршалом батареи был направлен на фронт. Феврале месяце, проезжая Черемхово, 

In [ ]:
for x in text_collection

In [ ]:
{'_id': ObjectId('64900ae9397b9020042fa347'), 
 'text': '  Милиционеру сказали : « С тобой будем разбираться , верни деньги » .   Вернули деньги ей .  ',
 'label': '+r', 
 'end': 1963,
 'begin': 1881, 
 'id_doc': ObjectId('64900ae9397b9020042fa32b')}

In [ ]:
[{'$or': [{'tags': {'$regex': '+s1'}}]}, 
 {'$or': [{'tags': {'$regex': '+s2'}}]},
 {'$or': [{'tags': {'$regex': '+s3'}}]}]

In [ ]:
[{'$or': [{'tags': {'$regex': 'hw'}}, 
          {'tags': {'$regex': 'dw'}}, 
          {'tags': {'$regex': 'vw'}}]}, 
 {'$or': [{'tags': {'$regex': '+s3'}}]}]

In [ ]:
[{'token': 'горе'}, 
 {'iniForm': 'гора'}, 
 {'$and': [{'$or': [{'tags': {'$regex': 'NOUN'}}, 
                    {'tags': {'$regex': 'ADJF'}}]}, 
           {'$or': [{'tags': {'$regex': 'nomn'}}, 
                    {'tags': {'$regex': 'loct'}}]}]}]

In [ ]:
[{'$and': [{'$or': [{'tags': {'$regex': 'hw'}}, 
                    {'tags': {'$regex': 'dw'}}, 
                    {'tags': {'$regex': 'vw'}}]}]}, 
 {'$and': [{'$or': [{'tags': {'$regex': '+s3'}}]}]}]

In [103]:
for x in dictionary.find({'token': 'горе'}).limit(100):
    print(x)

{'_id': ObjectId('64902f34f179e19ef227d40f'), 'token': 'горе', 'iniForm': 'гора', 'tags': 'NOUN,inan,femn sing,loct', 'freq': 1, 'docs': [ObjectId('64902f34f179e19ef227d40d')]}


In [110]:

#     {'token': 'горе'}, {'$and': [{'$or': [{'tags': {'$regex': 'NOUN'}}, {'tags': {'$regex': 'ADJF'}}, {'tags': {'$regex': 'nomn'}}, {'tags': {'$regex': 'acc'}}]}]}

In [176]:
# агрегация с условием больше чем по 1 полю

for y in plots.aggregate([{'$match' : {'$or': 
                                       [{'label': 'b4'}, {'label': '-s3'}]
                                      }}, 
                           {'$group': {'_id': '$id_doc'}}]):
    print(y)
    print(text_collection.find({"_id": y['_id']})[0]['doc_text'])

{'_id': ObjectId('64900af8397b9020042fa789')}
Мне мой милый изменил,
На козе уехал в Крым,
А я маху не дала,
На корове догнала.
{'_id': ObjectId('64900af8397b9020042fa79f')}
Без меня меня женили,
Я на мельнице был,
Приезжаю домой
Поздравляют с женой.


In [229]:
def get_docs(id_docs):
    res = []
    for doc in text_collection.find({'_id': {'$in': id_docs}}):
        ids = [r['_id'] for r in res if r]
        if doc['_id'] not in ids:
            res.append(doc)
    return res
# print(res)
res = get_docs(get_doc_ids('старый', [], [], []))
res

[{'_id': ObjectId('648cfa9161b224ea1ad2d616'),
  'doc_text': 'Я иду, а мне навстречу\nИдёт милка парою.\nВидно он провожает\nУхажёрку старую. ',
  'genre': 'Частушка, лирическая (встреча влюбленных).',
  'info': 'Плакида Варвара Григорьевна.',
  'comment': 'Диалектизмы: милка\nФеминитив: ухажёрка.',
  'collector': nan,
  'typewriter': nan},
 {'_id': ObjectId('648cfa9561b224ea1ad2d6f8'),
  'doc_text': 'Не завешивай окошки\nСтарой занавескою.\nВсё-равно мне не бывать\nВашею невесткою.',
  'genre': 'Частушки',
  'info': 'Лагутина Клавдия Николаевна\nНациональность: русская\nГод рождения: 1928г.\nМесто рождения: с Бальзино Дульдургинского р-на Читинской обл.\nОбразование: 4 класса начальной школы\nПрофессия: повар',
  'comment': nan,
  'collector': 'студентами 1 курса 3 группы филологического ф – та ИГУ Съяновой А., Тирских Н. 13 июля 1981г. Место записи: с. Бальзино Дульдургинского р – на Читинской обл.',
  'typewriter': 'студентка 1 курса группы 211121 – ЗБ  филологического ф-та ИГУ Иван

In [132]:
len(set(res))

4

In [125]:
# вывести тексты, сожержащие лемму "старая"
import re
res = []
lemma = 'старый'
response_tags = ['ед']
list_of_tags = []
if response_tags:
    list_of_tags = [{'forms.tags': re.compile(i)} for i in response_tags]
if lemma: 
    list_of_tags.append({'iniForm': preprocess(lemma)})
if list_of_tags:
    for x in lemmas.find({'$and': list_of_tags}):#, 'forms': {'$in': }'ПРИЛ,кач жр,ед,вн'}):
        print(x)
        for doc in text_collection.find({'_id': {'$in': x['docs']}}):
            res.append(doc['doc_text'])
#     for doc in text_collection.find({'_id': {'$in': x['docs']}}):
#         print(doc)
print(res)
#     print('-----------')

{'_id': ObjectId('648cfa9161b224ea1ad2d621'), 'iniForm': 'старый', 'POS': 'ADJF', 'freq': 8, 'docs': [ObjectId('648cfa9161b224ea1ad2d616'), ObjectId('648cfa9561b224ea1ad2d6f8'), ObjectId('648cfaa861b224ea1ad2da35'), ObjectId('648cfaab61b224ea1ad2dba1'), ObjectId('648f7dd9adadd388c446841b'), ObjectId('648f7de0adadd388c446844a'), ObjectId('648f7e08adadd388c4468518'), ObjectId('648f7e18adadd388c446859c')], 'forms': [{'token': 'старую', 'tags': 'ПРИЛ,кач жр,ед,вн'}, {'token': 'Старой', 'tags': 'ПРИЛ,кач жр,ед,рд'}, {'token': 'старый', 'tags': 'ПРИЛ,кач мр,ед,им'}, {'token': 'Старый', 'tags': 'ПРИЛ,кач мр,ед,им'}]}
['Я иду, а мне навстречу\nИдёт милка парою.\nВидно он провожает\nУхажёрку старую. ', 'Не завешивай окошки\nСтарой занавескою.\nВсё-равно мне не бывать\nВашею невесткою.', 'Когда мне было лет 17\nЯ полюбила старика.\nСтарик высокий, черноокий,\nА я как розочка цвела.\nСтарик дарил же мне колечко,\nСтарик дарил же мне браслет\nИ долго-долго мы встречались,\nНе знала мать, не знал о

In [157]:
def get_by_plot():
    for y in plots.aggregate([{'$match' : {'hero_type': {'$in': ['vz', 'pa']}}}, {'$group': {'_id': '$id_doc'}}]):
        print(text_collection.find({'_id': y['_id']})[0])

In [84]:
lemmas['forms']['tags'][0]

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, unicode_decode_error_handler='ignore'), 'mongo_for_folklore'), 'lemmas.forms.tags.0')

In [81]:
for x in lemmas.find({'forms.tags': 'ПРИЛ,кач жр,ед,вн'}):
    print(x)

{'_id': ObjectId('648cfa8a61b224ea1ad2d4e9'), 'iniForm': 'восточный', 'POS': 'ADJF', 'freq': 8, 'docs': [ObjectId('648cfa8661b224ea1ad2d3f4'), ObjectId('648f7dc4adadd388c44683d9')], 'forms': [{'token': 'Восточную', 'tags': 'ПРИЛ,кач жр,ед,вн'}]}
{'_id': ObjectId('648cfa8f61b224ea1ad2d5b6'), 'iniForm': 'новый', 'POS': 'ADJF', 'freq': 6, 'docs': [ObjectId('648cfa8f61b224ea1ad2d5a9'), ObjectId('648cfa9861b224ea1ad2d7cf'), ObjectId('648cfaab61b224ea1ad2dba1'), ObjectId('648f7dd5adadd388c44683fa'), ObjectId('648f7de6adadd388c446848a'), ObjectId('648f7e18adadd388c446859c')], 'forms': [{'token': 'новый', 'tags': 'ПРИЛ,кач неод,мр,ед,вн'}, {'token': 'новую', 'tags': 'ПРИЛ,кач жр,ед,вн'}, {'token': 'новых', 'tags': 'ПРИЛ,кач мн,рд'}]}
{'_id': ObjectId('648cfa8f61b224ea1ad2d5b8'), 'iniForm': 'большой', 'POS': 'ADJF', 'freq': 34, 'docs': [ObjectId('648cfa8f61b224ea1ad2d5a9'), ObjectId('648cfa9261b224ea1ad2d664'), ObjectId('648cfa9861b224ea1ad2d7cf'), ObjectId('648cfaac61b224ea1ad2dc08'), ObjectId

In [170]:
# вывести все примеры сюжета b4

for x in plots.find({'label': 'b4'}):
    print(text_collection.find({"_id": x['id_doc']})[0]['doc_text'][x['begin']:x['end']])
    print('-----------')

Без меня меня женили,
Я на мельнице был,
-----------


In [163]:
res = []
response_heroes = ['wm']
list_of_tags = []
if response_heroes:
    list_of_tags = [{'$match' : {'hero_type': i}} for i in response_heroes]



for y in plots.aggregate([{'$match' : {'hero_type': {'$in': ['vz', 'pa']}}}, {'$group': {'_id': '$id_doc'}}]):
    print(text_collection.find({'_id': y['_id']})[0]['_id'])
    print('-----')

648f7dfeadadd388c44684e1
-----
648cfab161b224ea1ad2dd8e
-----
648f7df6adadd388c446848b
-----
648cfaa461b224ea1ad2d972
-----
648cfaa061b224ea1ad2d8a0
-----
648f7e31adadd388c4468621
-----
648f7e48adadd388c446865e
-----
648cfab661b224ea1ad2deb4
-----


In [381]:
# вывести тексты в которых есть герои типа wm 

for y in plots.aggregate([{'$match' : {'hero_type': 'wm'}}, {'$group': {'_id': '$id_doc'}}]):
    print(text_collection.find({'_id': y['_id']})[0]['doc_text'])
    print('-----')

Мы с милёночком стояли,
Вечерком под уголком.
Продымилась моя кофточка
Зелёным табаком.
-----
Я иду, а мне навстречу
Идёт милка парою.
Видно он провожает
Ухажёрку старую. 
-----
Наши горы, горы, горы,
Горы с перекатами.
Хороши наши ребята
Как не наши женихи.
-----
Через речку быструю
Я мосточек выстрою:
Ходи милый, ходи мой,
Ходи летом и зимой!
-----


In [382]:
# вывести сколько всего персонажей есть в документе и  отсортировать
for x in plots.aggregate([{'$group' : {'_id': '$_id', 'count_pers': {'$max': '$hero_ind'}}}, {'$sort':{'count_pers':-1}}]):
    print(x)

{'_id': ObjectId('648ababf9fa8ca9c849f1d73'), 'count_pers': '2'}
{'_id': ObjectId('648ababf9fa8ca9c849f1d71'), 'count_pers': '1'}
{'_id': ObjectId('648abac09fa8ca9c849f1dbb'), 'count_pers': '1'}
{'_id': ObjectId('648abac09fa8ca9c849f1dd8'), 'count_pers': '1'}
{'_id': ObjectId('648ababf9fa8ca9c849f1d72'), 'count_pers': '1'}
{'_id': ObjectId('648abac09fa8ca9c849f1db8'), 'count_pers': '0'}
{'_id': ObjectId('648abac09fa8ca9c849f1de6'), 'count_pers': '0'}
{'_id': ObjectId('648ababf9fa8ca9c849f1d70'), 'count_pers': '0'}
{'_id': ObjectId('648abac09fa8ca9c849f1db7'), 'count_pers': '0'}
{'_id': ObjectId('648abac09fa8ca9c849f1db9'), 'count_pers': '0'}
{'_id': ObjectId('648abac09fa8ca9c849f1dd7'), 'count_pers': '0'}
{'_id': ObjectId('648abac09fa8ca9c849f1db6'), 'count_pers': '0'}
{'_id': ObjectId('648abac09fa8ca9c849f1df7'), 'count_pers': '0'}
{'_id': ObjectId('648abac09fa8ca9c849f1dba'), 'count_pers': '0'}
{'_id': ObjectId('648abac09fa8ca9c849f1dd9'), 'count_pers': None}
{'_id': ObjectId('648aba

In [44]:
# вывести все токены в единственном числе (поиск по грам. признакам)

for x in lemmas.find({'forms.tags':{'$regex':'nom'}}, {'forms.token', 'forms.tags'}):
    print(x)
    print('-----')
    break

{'_id': ObjectId('648ffed1397b9020042f9712'), 'forms': [{'token': 'Двадцать', 'tags': 'NUMR nomn'}]}
-----


In [45]:
for x in lemmas.find({'forms.tags':{'$regex':'acc'}}, {'forms.token', 'forms.tags'}):
    print(x)
    print('-----')
    break

{'_id': ObjectId('648ffed1397b9020042f9717'), 'forms': [{'token': 'года', 'tags': 'NOUN,inan,masc sing,gent'}, {'token': 'году', 'tags': 'NOUN,inan,masc sing,loc2'}, {'token': 'год', 'tags': 'NOUN,inan,masc sing,accs'}]}
-----


In [384]:
# lemmas.find()[0]

In [385]:
def find_NER(text):
    markup = ner(text)
    return markup

In [386]:
# проставляю географию
# ищу локации, создаю отдельное полее с координатами

for x in text_collection.find({'info' : {'$ne':''}}, {'info'}).limit(100):
    for span in find_NER(str(x['info'])).spans:
        if span.type == 'LOC':
            location = geolocator.geocode(x['info'][span.start:span.stop])
            if location != None:
                if text_collection.count_documents({'_id': x['_id'], 'coord':{'$exists':True}}) == 0:
                    text_collection.update_one({'_id': x['_id']}, 
                                                         {'$set':{'coord':{'type': 'Point', 'coordinates': [round(location.latitude, 8), round(location.longitude, 8)]}}})

                    print(text_collection.find({'_id': x['_id']})[0])    

{'_id': ObjectId('648ababf9fa8ca9c849f1d91'), 'doc_text': 'Там под старым дубом\nПартизан лежит.\nЗолотые кудри\nВетер шевелит.\n\nОн лежит не дышит,\nА как – будто спит.\nПеред ним старушка\nМать его стоит.\n\nСлёзы проливает,\nТихо говорит:\n«Я тебя растила, нежно берегла,\nА теперь могила будет здесь твоя.»', 'genre': 'Лирическая песня\nПартизанская песня\n«Печальная»', 'info': 'Матвеев Анатолий Николаевич\nНациональность: русский\nГод рождения: 1932г.\nМесто рождения: деревня Алексино Псковской обл.\nОбразование: средне-специальное\nПрофессия: механик', 'comment': nan, 'collector': 'студентами 1 курса 3 группы филологического ф – та ИГУ Съяновой А., Тирских Н. 13 июля 1981г. Место записи: с. Бальзино Дульдургинского р – на Читинской обл.', 'typewriter': 'студентка 1 курса группы 211121 – ЗБ  филологического ф-та ИГУ Иванова Ульяна 2019 г. Место: птг. Тыреть, мкр Солерудник.', 'coord': {'type': 'Point', 'coordinates': [56.2340082, 38.4989335]}}


In [387]:
text_collection.create_index([("coord", pymongo.GEOSPHERE)], unique=False)

'coord_2dsphere'

In [388]:
for x in text_collection.find({'coord':{'$exists':True}}):
    print(x)
    print('--------')
text_collection.count_documents({'coord':{'$exists':True}})

{'_id': ObjectId('648ababf9fa8ca9c849f1d91'), 'doc_text': 'Там под старым дубом\nПартизан лежит.\nЗолотые кудри\nВетер шевелит.\n\nОн лежит не дышит,\nА как – будто спит.\nПеред ним старушка\nМать его стоит.\n\nСлёзы проливает,\nТихо говорит:\n«Я тебя растила, нежно берегла,\nА теперь могила будет здесь твоя.»', 'genre': 'Лирическая песня\nПартизанская песня\n«Печальная»', 'info': 'Матвеев Анатолий Николаевич\nНациональность: русский\nГод рождения: 1932г.\nМесто рождения: деревня Алексино Псковской обл.\nОбразование: средне-специальное\nПрофессия: механик', 'comment': nan, 'collector': 'студентами 1 курса 3 группы филологического ф – та ИГУ Съяновой А., Тирских Н. 13 июля 1981г. Место записи: с. Бальзино Дульдургинского р – на Читинской обл.', 'typewriter': 'студентка 1 курса группы 211121 – ЗБ  филологического ф-та ИГУ Иванова Ульяна 2019 г. Место: птг. Тыреть, мкр Солерудник.', 'coord': {'type': 'Point', 'coordinates': [56.2340082, 38.4989335]}}
--------


1

In [389]:
request=text_collection.find(
{
   "coord": {
     "$near": {
          "coordinates": [56.2340082, 38.4989335]
       },
       "$maxDistance": 11,
       "$minDistance":0,
     }
   }, {}
)
#req
for x in request:    
    print(x)

{'_id': ObjectId('648ababf9fa8ca9c849f1d91'), 'doc_text': 'Там под старым дубом\nПартизан лежит.\nЗолотые кудри\nВетер шевелит.\n\nОн лежит не дышит,\nА как – будто спит.\nПеред ним старушка\nМать его стоит.\n\nСлёзы проливает,\nТихо говорит:\n«Я тебя растила, нежно берегла,\nА теперь могила будет здесь твоя.»', 'genre': 'Лирическая песня\nПартизанская песня\n«Печальная»', 'info': 'Матвеев Анатолий Николаевич\nНациональность: русский\nГод рождения: 1932г.\nМесто рождения: деревня Алексино Псковской обл.\nОбразование: средне-специальное\nПрофессия: механик', 'comment': nan, 'collector': 'студентами 1 курса 3 группы филологического ф – та ИГУ Съяновой А., Тирских Н. 13 июля 1981г. Место записи: с. Бальзино Дульдургинского р – на Читинской обл.', 'typewriter': 'студентка 1 курса группы 211121 – ЗБ  филологического ф-та ИГУ Иванова Ульяна 2019 г. Место: птг. Тыреть, мкр Солерудник.', 'coord': {'type': 'Point', 'coordinates': [56.2340082, 38.4989335]}}


In [390]:
geolocator.reverse([56.2340082, 38.4989335])

Location(33А, Заречная улица, Алексино, Каринское сельское поселение, Александровский район, Владимирская область, Центральный федеральный округ, Россия, (56.2336904, 38.499001925, 0.0))

In [391]:
request=text_collection.find(
{
   "coord": {
     "$near": {
          "coordinates": [geolocator.geocode('Центральный федеральный округ').latitude,
                          geolocator.geocode('Центральный федеральный округ').longitude]
       },
       "$maxDistance": 1000000,
       "$minDistance":0,
     }
   }, {}
)
#req
for x in request:    
    print(x)

{'_id': ObjectId('648ababf9fa8ca9c849f1d91'), 'doc_text': 'Там под старым дубом\nПартизан лежит.\nЗолотые кудри\nВетер шевелит.\n\nОн лежит не дышит,\nА как – будто спит.\nПеред ним старушка\nМать его стоит.\n\nСлёзы проливает,\nТихо говорит:\n«Я тебя растила, нежно берегла,\nА теперь могила будет здесь твоя.»', 'genre': 'Лирическая песня\nПартизанская песня\n«Печальная»', 'info': 'Матвеев Анатолий Николаевич\nНациональность: русский\nГод рождения: 1932г.\nМесто рождения: деревня Алексино Псковской обл.\nОбразование: средне-специальное\nПрофессия: механик', 'comment': nan, 'collector': 'студентами 1 курса 3 группы филологического ф – та ИГУ Съяновой А., Тирских Н. 13 июля 1981г. Место записи: с. Бальзино Дульдургинского р – на Читинской обл.', 'typewriter': 'студентка 1 курса группы 211121 – ЗБ  филологического ф-та ИГУ Иванова Ульяна 2019 г. Место: птг. Тыреть, мкр Солерудник.', 'coord': {'type': 'Point', 'coordinates': [56.2340082, 38.4989335]}}


In [392]:
# проверяем относится ли к центральному федеральному округу установленная точка 
# (правильный ответ да)

for d in text_collection.find({'coord':{'$geoWithin':{"$box": 
[[50.72646354315088,31.345877414798924],[57.77272308914989,42.36177531910842]]
}}}):
    print(d)

{'_id': ObjectId('648ababf9fa8ca9c849f1d91'), 'doc_text': 'Там под старым дубом\nПартизан лежит.\nЗолотые кудри\nВетер шевелит.\n\nОн лежит не дышит,\nА как – будто спит.\nПеред ним старушка\nМать его стоит.\n\nСлёзы проливает,\nТихо говорит:\n«Я тебя растила, нежно берегла,\nА теперь могила будет здесь твоя.»', 'genre': 'Лирическая песня\nПартизанская песня\n«Печальная»', 'info': 'Матвеев Анатолий Николаевич\nНациональность: русский\nГод рождения: 1932г.\nМесто рождения: деревня Алексино Псковской обл.\nОбразование: средне-специальное\nПрофессия: механик', 'comment': nan, 'collector': 'студентами 1 курса 3 группы филологического ф – та ИГУ Съяновой А., Тирских Н. 13 июля 1981г. Место записи: с. Бальзино Дульдургинского р – на Читинской обл.', 'typewriter': 'студентка 1 курса группы 211121 – ЗБ  филологического ф-та ИГУ Иванова Ульяна 2019 г. Место: птг. Тыреть, мкр Солерудник.', 'coord': {'type': 'Point', 'coordinates': [56.2340082, 38.4989335]}}
